In [1]:
# we scraper
from bs4 import BeautifulSoup

# request
import requests

# io
from io import BytesIO

# date parser and sleep
from dateutil.parser import parse
import time

# web driver
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

# reeading and exporting to excel files
import pandas as pd

# looping process monitoring
from tqdm import tqdm
import random

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

s = slice(11)

In [42]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("SEALAND")

In [44]:
len(bl_list)

34

In [45]:
bl_list

['227145963',
 '227927058',
 '228000484',
 '228698779',
 '228667042',
 '228487087',
 '228478181',
 '228495628',
 '228528205',
 '228576712',
 '228151079',
 '228326741',
 '228329054',
 '228776329',
 '228627103',
 '228107228',
 '228404249',
 '228422054',
 '228876620',
 '228883601',
 '228877614',
 '229154632',
 '229327088',
 '231758967',
 '231844017',
 '231838198',
 '231961328',
 '231961336',
 '231877995',
 '231935360',
 '232000963',
 '232069300',
 '232079773',
 '232095269']

In [46]:
dict_hasil = {}
current_dict = {}
list_dict = []
gagal = []

In [38]:
bl_list = ['231877995', '227145963']

**FIX**


In [47]:
list_dict = []
gagal = []
# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome(use_subprocess=True)
driver.get('https://www.maersk.com/tracking/' + bl_list[random.randint(0, len(bl_list)-1)])

# wait to load all
time.sleep(2)

#click allow all
click_allowall = driver.find_element(By.XPATH,'//button[text()="Allow all"]')
click_allowall.click()

for i, bls in enumerate(tqdm(bl_list)):
    try:
        # access the web
        driver.get('https://www.maersk.com/tracking/' + bl_list[i])
        # sleep wait
        title = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@id='maersk-app']/div/div/div/dl")))
        time.sleep(3)
        
        # finding how many hide details in 1 page refering to how many container number in 1 bl
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        containers = soup1.find_all('div', {"data-test": "container"})
        number_of_ctr = len(containers)
        
        print(bls, "HAVE ", number_of_ctr, " CTR")
        # print(dict_hasil)

        if number_of_ctr > 1:
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                #
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            print("Hasil dari Append Judul")
            print(dict_hasil)
            if driver.find_element(By.CLASS_NAME, "track__feedback"):
                x=4
            else:
                x=3
            # clicking all view more
            for c in range(number_of_ctr):
                try:
                    print("clicking div no :",  x)
                    show_details2 = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/div/div/div[{}]/dl/button'.format(x))))  
                    # /html/body/main/div/div/div/div/div[3]/dl/button/span
                    show_details_check = driver.find_element(By.XPATH, "/html/body/main/div/div/div/div/div[{}]/dl/button/span".format(x))
                    if show_details_check.text == 'Show details':
                        show_details2.click()
                    print(show_details_check.text)
                    
                    time.sleep(1)
                except Exception as e:
                    print(e)
                x=x+1

            time.sleep(2)
            # taking the whole page's html
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)
                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
                 
                for panjang in range(0, len(milestones), 2):
                    key = milestones[panjang]
                    value = milestones[panjang + 1]

                    if key == 'Container Number':
                        if current_dict:
                            list_dict.append(current_dict.copy())
                        current_dict = {'Container Number': value}
                    else:
                        current_dict[key] = value

                    current_dict.update(dict_hasil) 
                print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
            print(bls, ", completed!!")
        else:
            time.sleep(1)
            
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            time.sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)

                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
            
            for t, milestone in enumerate(milestones):
                if milestone == "Container Number":
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                elif t%2==0 :
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                    
            current_dict.update(dict_hasil) 
            print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
        
        unique_list = []
        for d in list_dict:
            if d not in unique_list:
                unique_list.append(d)
                
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/34 [00:00<?, ?it/s]

227145963 HAVE  1  CTR
['Bill of Lading number', '227145963', 'From', 'Jakarta', 'To', 'Manila']


  3%|▎         | 1/34 [00:09<05:07,  9.31s/it]

{'Container Number': 'SUDU8975898', 'Jakarta Gate outEmpty': '01 May 2023', 'Jakarta Gate in': '02 May 2023', 'Jakarta Load': '04 May 2023', 'Jakarta Vessel departure': '05 May 2023', 'Bill of Lading number': '227145963', 'From': 'Jakarta', 'To': 'Manila'}
227927058 HAVE  2  CTR
['Bill of Lading number', '227927058', 'From', 'Jakarta', 'To', 'Subic']
Hasil dari Append Judul
{'Bill of Lading number': '227927058', 'From': 'Jakarta', 'To': 'Subic'}
clicking div no : 4
Hide details
clicking div no : 5
Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF729A88EF2+54786]
	(No symbol) [0x00007FF7299F5612]
	(No symbol) [0x00007FF7298AA64B]
	(No symbol) [0x00007FF7298EB79C]
	(No symbol) [0x00007FF7298EB91C]
	(No symbol) [0x00007FF729926D87]
	(No symbol) [0x00007FF72990BEAF]
	(No symbol) [0x00007FF729924D02]
	(No symbol) [0x00007FF72990BC43]
	(No symbol) [0x00007FF7298E0941]
	(No symbol) [0x00007FF7298E1B84]
	GetHandleVerifier [0x00007FF729DD7F52+3524194]
	GetHandleVerifier [0x00007FF729E2D800+38

  6%|▌         | 2/34 [00:50<15:05, 28.31s/it]

...

{'Container Number': 'SUDU8861766', 'Jakarta Gate outEmpty': '10 Oct 2023', 'Jakarta Gate in': '11 Oct 2023', 'Jakarta Load': '16 Oct 2023', 'Jakarta Vessel departure': '16 Oct 2023', 'Tanjung Pelepas Vessel arrival': '19 Oct 2023', 'Tanjung Pelepas Discharge': '19 Oct 2023', 'Tanjung Pelepas Load': '24 Oct 2023', 'Tanjung Pelepas Vessel departure': '25 Oct 2023', 'Kuching Vessel arrival': '26 Oct 2023', 'Kuching Discharge': '28 Oct 2023', 'Kuching Gate out': '30 Oct 2023', 'Bill of Lading number': '232069300', 'From': 'Jakarta', 'To': 'Kuching'}
232079773 HAVE  1  CTR
['Bill of Lading number', '232079773', 'From', 'Jakarta', 'To', 'Sibu']


 97%|█████████▋| 33/34 [14:49<00:17, 17.79s/it]

{'Container Number': 'TRHU7549835', 'Jakarta Gate outEmpty': '10 Oct 2023', 'Jakarta Gate in': '13 Oct 2023', 'Jakarta Load': '16 Oct 2023', 'Jakarta Vessel departure': '16 Oct 2023', 'Tanjung Pelepas Vessel arrival': '19 Oct 2023', 'Tanjung Pelepas Discharge': '19 Oct 2023', 'Tanjung Pelepas Load': '21 Oct 2023', 'Tanjung Pelepas Vessel departure': '21 Oct 2023', 'Bintulu Vessel arrival': '25 Oct 2023', 'Bintulu Discharge': '27 Oct 2023', 'Bill of Lading number': '232079773', 'From': 'Jakarta', 'To': 'Sibu'}
232095269 HAVE  1  CTR
['Bill of Lading number', '232095269', 'From', 'Jakarta', 'To', 'Tanjung Pelepas']


100%|██████████| 34/34 [15:00<00:00, 26.48s/it]

{'Container Number': 'SEKU4465000', 'Jakarta Gate outEmpty': '12 Oct 2023', 'Jakarta Gate in': '14 Oct 2023', 'Jakarta Load': '20 Oct 2023', 'Jakarta Vessel departure': '20 Oct 2023', 'Tanjung Pelepas Vessel arrival': '22 Oct 2023', 'Tanjung Pelepas Discharge': '22 Oct 2023', 'Tanjung Pelepas Gate out': '26 Oct 2023', 'Tanjung Pelepas Empty container return': '30 Oct 2023', 'Bill of Lading number': '232095269', 'From': 'Jakarta', 'To': 'Tanjung Pelepas'}


In [48]:
unique_list

[{'Container Number': 'SUDU8975898',
  'Jakarta Gate outEmpty': '01 May 2023',
  'Jakarta Gate in': '02 May 2023',
  'Jakarta Load': '04 May 2023',
  'Jakarta Vessel departure': '05 May 2023',
  'Bill of Lading number': '227145963',
  'From': 'Jakarta',
  'To': 'Manila'},
 {'Container Number': 'MRSU4884755',
  'Bill of Lading number': '227927058',
  'From': 'Jakarta',
  'To': 'Subic',
  'Jakarta Gate outEmpty': '31 May 2023',
  'Jakarta Gate in': '03 Jun 2023',
  'Jakarta Load': '09 Jun 2023',
  'Jakarta Vessel departure': '09 Jun 2023',
  'Subic Vessel arrival': '21 Jun 2023',
  'Subic Discharge': '22 Jun 2023'},
 {'Container Number': 'MRSU5987887',
  'Bill of Lading number': '227927058',
  'From': 'Jakarta',
  'To': 'Subic',
  'Jakarta Gate outEmpty': '30 May 2023',
  'Jakarta Gate in': '03 Jun 2023',
  'Jakarta Load': '09 Jun 2023',
  'Jakarta Vessel departure': '09 Jun 2023',
  'Subic Vessel arrival': '21 Jun 2023',
  'Subic Discharge': '22 Jun 2023'},
 {'Container Number': 'CAAU65

In [49]:
filtered_dict_list =[]
for dict_milestone in unique_list:
    # print(dict_milestone)
    kw = ['Bill of Lading number', 'Container Number', 'From', 'To', dict_milestone['From'], dict_milestone['To']]
    dict_key = []
    for keys in dict_milestone:
        dict_key.append(keys)
    # print("BEFORE", dict_key)
    new_list = [] 
    
    for key in dict_key:
        if any(substring in key for substring in kw):
            new_list.append(key)
    # print("AFTER",new_list)
    dict_milestone = {key: dict_milestone[key] for key in new_list}
    filtered_dict_list.append(dict_milestone)

In [50]:
filtered_dict_list

[{'Container Number': 'SUDU8975898',
  'Jakarta Gate outEmpty': '01 May 2023',
  'Jakarta Gate in': '02 May 2023',
  'Jakarta Load': '04 May 2023',
  'Jakarta Vessel departure': '05 May 2023',
  'Bill of Lading number': '227145963',
  'From': 'Jakarta',
  'To': 'Manila'},
 {'Container Number': 'MRSU4884755',
  'Bill of Lading number': '227927058',
  'From': 'Jakarta',
  'To': 'Subic',
  'Jakarta Gate outEmpty': '31 May 2023',
  'Jakarta Gate in': '03 Jun 2023',
  'Jakarta Load': '09 Jun 2023',
  'Jakarta Vessel departure': '09 Jun 2023',
  'Subic Vessel arrival': '21 Jun 2023',
  'Subic Discharge': '22 Jun 2023'},
 {'Container Number': 'MRSU5987887',
  'Bill of Lading number': '227927058',
  'From': 'Jakarta',
  'To': 'Subic',
  'Jakarta Gate outEmpty': '30 May 2023',
  'Jakarta Gate in': '03 Jun 2023',
  'Jakarta Load': '09 Jun 2023',
  'Jakarta Vessel departure': '09 Jun 2023',
  'Subic Vessel arrival': '21 Jun 2023',
  'Subic Discharge': '22 Jun 2023'},
 {'Container Number': 'CAAU65

In [51]:
list_of_dict_fix = []
for filter_dict in filtered_dict_list:
    replacement_mapping = {
        filter_dict["From"]: 'Origin',
        filter_dict["To"]: 'Destination'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

    list_of_dict_fix.append(updated_dict)

In [52]:
list_of_dict_fix

[{'Container Number': 'SUDU8975898',
  'Origin Gate outEmpty': '01 May 2023',
  'Origin Gate in': '02 May 2023',
  'Origin Load': '04 May 2023',
  'Origin Vessel departure': '05 May 2023',
  'Bill of Lading number': '227145963',
  'From': 'Jakarta',
  'To': 'Manila'},
 {'Container Number': 'MRSU4884755',
  'Bill of Lading number': '227927058',
  'From': 'Jakarta',
  'To': 'Subic',
  'Origin Gate outEmpty': '31 May 2023',
  'Origin Gate in': '03 Jun 2023',
  'Origin Load': '09 Jun 2023',
  'Origin Vessel departure': '09 Jun 2023',
  'Destination Vessel arrival': '21 Jun 2023',
  'Destination Discharge': '22 Jun 2023'},
 {'Container Number': 'MRSU5987887',
  'Bill of Lading number': '227927058',
  'From': 'Jakarta',
  'To': 'Subic',
  'Origin Gate outEmpty': '30 May 2023',
  'Origin Gate in': '03 Jun 2023',
  'Origin Load': '09 Jun 2023',
  'Origin Vessel departure': '09 Jun 2023',
  'Destination Vessel arrival': '21 Jun 2023',
  'Destination Discharge': '22 Jun 2023'},
 {'Container Numb

In [53]:
list_of_dict_fix2 = []
for filter_dict in list_of_dict_fix:
    replacement_mapping = {
        "Bill of Lading number" : "BL Number",
        "Origin Vessel departure": 'ATD',
        "Destination Discharge": 'ATA',
        "Destination Gate out": 'Container Release',
        "Destination Empty container return" : 'Container Return'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

        if is_date(value):
           input_date = datetime.strptime(value, '%d %b %Y')
           updated_dict[key] = input_date.strftime("%Y-%m-%d")
    
    updated_dict["Liners"] = "SEALAND"

    list_of_dict_fix2.append(updated_dict)

In [54]:
list_of_dict_fix2

[{'Container Number': 'SUDU8975898',
  'Origin Gate outEmpty': '2023-05-01',
  'Origin Gate in': '2023-05-02',
  'Origin Load': '2023-05-04',
  'ATD': '2023-05-05',
  'BL Number': '227145963',
  'From': 'Jakarta',
  'To': 'Manila',
  'Liners': 'SEALAND'},
 {'Container Number': 'MRSU4884755',
  'BL Number': '227927058',
  'From': 'Jakarta',
  'To': 'Subic',
  'Origin Gate outEmpty': '2023-05-31',
  'Origin Gate in': '2023-06-03',
  'Origin Load': '2023-06-09',
  'ATD': '2023-06-09',
  'Destination Vessel arrival': '2023-06-21',
  'ATA': '2023-06-22',
  'Liners': 'SEALAND'},
 {'Container Number': 'MRSU5987887',
  'BL Number': '227927058',
  'From': 'Jakarta',
  'To': 'Subic',
  'Origin Gate outEmpty': '2023-05-30',
  'Origin Gate in': '2023-06-03',
  'Origin Load': '2023-06-09',
  'ATD': '2023-06-09',
  'Destination Vessel arrival': '2023-06-21',
  'ATA': '2023-06-22',
  'Liners': 'SEALAND'},
 {'Container Number': 'CAAU6577730',
  'Origin Gate outEmpty': '2023-06-01',
  'Origin Gate in':

In [55]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict_fix2)

Today's Collection Name ===>  all_tracking_Oct-31-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [56]:
gagal

[]

In [63]:
df = pd.DataFrame.from_dict(list_of_dict_fix2)
df.to_excel('sealand_maintained.xlsx')

In [16]:
list_dict = []
# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome(use_subprocess=True)
driver.get('https://www.sealandmaersk.com/tracking/' + bl_list[random.randint(0, len(bl_list)-1)])

# wait to load all
time.sleep(2)

#click allow all
click_allowall = driver.find_element(By.XPATH,'//button[text()="Allow all"]')
click_allowall.click()

# click asia
click_asia = driver.find_element(By.XPATH,"/html/body/div[2]/div/section/div/div[2]/form/div[1]/div/div/ol/li[3]/label/div[1]")
click_asia.click()

# click continue
click_continue = driver.find_element(By.ID,'confirmRegion')
click_continue.click()

for i, bls in enumerate(tqdm(gagal)):
    try:
        # access the web
        driver.get('https://www.sealandmaersk.com/tracking/' + bl_list[i])

        # sleep wait
        time.sleep(3)
        
        # finding how many hide details in 1 page refering to how many container number in 1 bl
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        containers = soup1.find_all('div', {"data-test": "container"})
        number_of_ctr = len(containers)
        
        print(bls, "HAVE ", number_of_ctr, " CTR")
        # print(dict_hasil)

        # print("current bl: ", bls, "\n number of container: ", number_of_ctr)
        if number_of_ctr > 1:
            # taking BL Number, From, and To 
            title = driver.find_element(By.XPATH, "/html/body/main/div/div/dl")
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                #
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            print("Hasil dari Append Judul")
            print(dict_hasil)
            x=3
            # clicking all view more
            for c in range(number_of_ctr):
                try:
                    print("clicking div no :",  x)
                    show_details2 = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/div[{}]/dl/button'.format(x))))  
                    show_details2.click()
                    time.sleep(1)
                except Exception as e:
                    print(e)
                x=x+1

            time.sleep(2)
            # taking the whole page's html
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)
                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
                 
                for panjang in range(0, len(milestones), 2):
                    key = milestones[panjang]
                    value = milestones[panjang + 1]

                    if key == 'Container Number':
                        if current_dict:
                            list_dict.append(current_dict.copy())
                        current_dict = {'Container Number': value}
                    else:
                        current_dict[key] = value

                    current_dict.update(dict_hasil) 
                print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
            print(bls, ", completed!!")
        else:
            time.sleep(1)
            
            title = driver.find_element(By.XPATH, "/html/body/main/div/div/dl")
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            for a, item in enumerate(titles):
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            time.sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)

                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
            
            for t, milestone in enumerate(milestones):
                if milestone == "Container Number":
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                elif t%2==0 :
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                    
            current_dict.update(dict_hasil) 
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
        
        unique_list = []
        for d in list_dict:
            if d not in unique_list:
                unique_list.append(d)

    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/1 [00:00<?, ?it/s]

231590994 HAVE  1  CTR


100%|██████████| 1/1 [00:08<00:00,  8.01s/it]


In [17]:
unique_list

[{'Container Number': 'CAAU6577730',
  'Jakarta Gate outEmpty': '01 Jun 2023',
  'Jakarta Gate in': '03 Jun 2023',
  'Jakarta Load': '09 Jun 2023',
  'Jakarta Vessel departure': '09 Jun 2023',
  'Subic Vessel arrival': '21 Jun 2023',
  'Subic Discharge': '22 Jun 2023',
  'Bill of Lading number': '228000484',
  'From': 'Jakarta',
  'To': 'Subic'}]

In [13]:
from datetime import datetime
#declarate get

date = datetime.today().strftime("%d.%m.%Y")
print(date)

11.10.2023
